In [1]:
import sys
import subprocess
import importlib

# ---- CONFIGURATION ----
REQUIRED_PYTHON = (3, 9)
REQUIRED_PACKAGES = [
    "pandas",
    "openpyxl",
    "sqlalchemy",
    "psycopg[binary]",
    "jupyterlab",
]

# ---- PYTHON VERSION CHECK ----
if sys.version_info < REQUIRED_PYTHON:
    raise SystemExit(
        f"❌ Python {REQUIRED_PYTHON[0]}.{REQUIRED_PYTHON[1]}+ required, "
        f"found {sys.version_info.major}.{sys.version_info.minor}"
    )
else:
    print(f"✅ Python version OK: {sys.version_info.major}.{sys.version_info.minor}")

# ---- PACKAGE CHECK / AUTO-INSTALL ----
def install(pkg):
    """Install a package via pip in the current environment."""
    print(f"⬇️  Installing missing package: {pkg}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

missing = []
for pkg in REQUIRED_PACKAGES:
    # psycopg[binary] isn't a real import name, so handle that separately
    import_name = pkg.split("[")[0]
    try:
        importlib.import_module(import_name)
    except ImportError:
        missing.append(pkg)

if missing:
    print(f"\n⚠️ Missing packages detected: {missing}")
    for pkg in missing:
        install(pkg)
else:
    print("✅ All required packages are already installed.")

# ---- SHOW INSTALLED VERSIONS ----
print("\n📦 Installed versions:")
for pkg in REQUIRED_PACKAGES:
    pkg_base = pkg.split("[")[0]
    try:
        mod = importlib.import_module(pkg_base)
        version = getattr(mod, "__version__", "unknown")
        print(f"  - {pkg_base} == {version}")
    except ImportError:
        print(f"  - {pkg_base} not installed")


✅ Python version OK: 3.10
✅ All required packages are already installed.

📦 Installed versions:
  - pandas == 2.3.3
  - openpyxl == 3.1.5
  - sqlalchemy == 2.0.44
  - psycopg == 3.2.12
  - jupyterlab == 4.4.10


In [2]:
!pip freeze > requirements.txt
print("✅ requirements.txt updated")

✅ requirements.txt updated


In [1]:
# --- CONFIG ---
from pathlib import Path

# PostgreSQL
PG_USER = "postgres"
PG_PASS = "greenage"
PG_HOST = "localhost"
PG_PORT = 5432
PG_DB   = "moin_weather"   # make sure DB is UTF8
PG_SCHEMA = "lgs2"
TABLE_NAME = "crop_varieties"

# Excel source
EXCEL_PATH = Path(r"./Crop Growth Models - Wheat (4).xlsx")
SHEET_NAME = "Crop_Norms"
USECOLS    = "B:CI"     # adjust if needed
HEADER_ROW = 3          # Excel row 4 -> pandas header=3 (0-indexed)

# --- ENGINE ---
from sqlalchemy import create_engine, text

conn_str = f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
# client_encoding can be SET after connect; psycopg3 also accepts options, but SET works reliably
engine = create_engine(conn_str, pool_pre_ping=True)

with engine.connect() as conn:
    server_enc = conn.execute(text("SHOW SERVER_ENCODING;")).scalar_one()
    conn.execute(text("SET client_encoding TO 'UTF8';"))
    client_enc = conn.execute(text("SHOW CLIENT_ENCODING;")).scalar_one()
    print("SERVER_ENCODING:", server_enc)
    print("CLIENT_ENCODING:", client_enc)


SERVER_ENCODING: UTF8
CLIENT_ENCODING: UTF8


In [2]:
import pandas as pd

# Read as strings first to keep full control over conversion
df_raw = pd.read_excel(
    EXCEL_PATH,
    sheet_name=SHEET_NAME,
    header=HEADER_ROW,
    usecols=USECOLS,
    dtype=str,
    engine="openpyxl",
)

# strip whitespace in *all* string cells
df_raw = df_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# drop fully empty rows
df_raw = df_raw.dropna(how="all")

print("Shape after load & trim:", df_raw.shape)
print("Raw columns:")
print(df_raw.columns.tolist())

display(df_raw.head())


Shape after load & trim: (26, 86)
Raw columns:
['Crop_Name', 'Variety_Name', 'Variety_Type', 'Height_Class', 'Height_cm', 'Nutrient_Content', 'Common_Disease_Tolerance', 'Common_Pest_Tolerance', 'Environmental_Resilience', 'Category', 'Country', 'Season', 'PAR', 'Region', 'Seed_Weight_mg', 'Crop_Min_Temp', 'Base_Temp', 'Opt._Temp', 'Upper_Temp', 'Min_Days', 'Max_Days', 'Ripening_S_GDD', 'Ripening_E_GDD', 'Avg. Daily GDD', 'Days_to_Ripening', 'Sunshine', 'Soil_Temp_Min', 'VPA_Max', 'Humidity_Max', 'Dew_Point_Min', 'WS_2m_Max', 'GDD_Adjust', 'pH_Min', 'pH_Max', 'CWR_Min', 'CWR_Max', 'Sowing_Depth_cm', 'Root_Depth_cm', 'Stress_Tolerance_Index', 'Row_Spacing_cm', 'Plant_to_Plant_cm', 'Seed_Per_Spot', 'Seed_Rate_m2', 'Seed_Weight_kg/ha', 'Germination_%', 'Effective_Plants_m2', 'Tillers_Per_Plant', 'Shoots_Per_m2', 'Survival_Ratio', 'Shoot_Bearing_m2', 'Leaf_Area_Per_Shoot_m2', 'Start_LAI', 'LAI_Start_GDD', 'Max_LAI', 'LAI_Max_GDD', 'End_LAI', 'LAI_End_GDD', 'RUE_Max', 'RUE_Asymptote_Frac', 

C:\Users\ali_raza\AppData\Local\Temp\ipykernel_22144\4073561252.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_raw = df_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Crop_Name,Variety_Name,Variety_Type,Height_Class,Height_cm,Nutrient_Content,Common_Disease_Tolerance,Common_Pest_Tolerance,Environmental_Resilience,Category,...,Sandy Loam,Loam,Silt Loam,Clay Loam,Silt,Sandy Clay,Silty Clay Loam,Silty Clay,Clay,Sandy Clay Loam
0,wheat,Sadiq-21,OPV,Medium,115,Protein 12-13%,Leaf & Yellow Rust (MR),Aphid (Moderate),Drought Resistance,Cereal,...,1,1,1,1,0,0,0,0,1,0
1,wheat,aari-2011,OPV,Medium,105,Protein 12-13%,"Leaf rust, stripe rust",Termite,NaN,Cereal,...,1,1,1,1,0,0,0,0,1,0
2,wheat,aas-2011,OPV,Medium,105,Protein 12-13%,Rust resistant,Aphid,NaN,Cereal,...,1,1,1,1,0,0,0,0,1,0
3,wheat,akbar-2019,OPV,Medium,105,Protein 12-13%,Rust resistant (all 3 rusts),Aphid,NaN,Cereal,...,1,1,1,1,0,0,0,0,1,0
4,wheat,anaj-2017,OPV,Medium,105,Protein 12-13%,Rust resistant,Aphid,NaN,Cereal,...,1,1,1,1,0,0,0,0,1,0


In [3]:
import re

def to_snake(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"[^\w\s]+", " ", s)  # punctuation -> spaces
    s = re.sub(r"\s+", "_", s)       # whitespace -> underscore
    return s.lower()

snake_map = {c: to_snake(c) for c in df_raw.columns}

# Apply initial snake-case
df_snake = df_raw.rename(columns=snake_map)

# --- Your custom overrides (keys are snake-case) ---
# e.g. raw "Crop Name" -> snake "crop_name" -> override to "crop_fk"
custom_overrides = {
    "crop_name": "crop_fk",
    "variety_name": "variety_eng",
    "ripening_s_gdd": "ripening_start_gdd",
    "ripening_e_gdd": "ripening_end_gdd",
    "germination__": "germination_percentage",
    "hi": "harvest_index",
    "crop_min_temp": "min_temp",
    "opt__temp": "opt_temp",
    "dm_fm": "dm_fm_ratio",
    }

# Apply overrides
rename_map = {**{v: v for v in df_snake.columns}, **custom_overrides}
df_renamed = df_snake.rename(columns=rename_map)

print("Column rename preview:")
display(pd.DataFrame({
    "original": list(df_raw.columns),
    "snake": [snake_map[c] for c in df_raw.columns],
    "final": [rename_map.get(snake_map[c], snake_map[c]) for c in df_raw.columns],
}))

print("Renamed columns:")
print(df_renamed.columns.tolist())
display(df_renamed.head())


Column rename preview:


,original,snake,final
0,Crop_Name,crop_name,crop_fk
1,Variety_Name,variety_name,variety_eng
2,Variety_Type,variety_type,variety_type
3,Height_Class,height_class,height_class
4,Height_cm,height_cm,height_cm
...,...,...,...
81,Sandy Clay,sandy_clay,sandy_clay
82,Silty Clay Loam,silty_clay_loam,silty_clay_loam
83,Silty Clay,silty_clay,silty_clay
84,Clay,clay,clay


Renamed columns:
['crop_fk', 'variety_eng', 'variety_type', 'height_class', 'height_cm', 'nutrient_content', 'common_disease_tolerance', 'common_pest_tolerance', 'environmental_resilience', 'category', 'country', 'season', 'par', 'region', 'seed_weight_mg', 'min_temp', 'base_temp', 'opt_temp', 'upper_temp', 'min_days', 'max_days', 'ripening_start_gdd', 'ripening_end_gdd', 'avg_daily_gdd', 'days_to_ripening', 'sunshine', 'soil_temp_min', 'vpa_max', 'humidity_max', 'dew_point_min', 'ws_2m_max', 'gdd_adjust', 'ph_min', 'ph_max', 'cwr_min', 'cwr_max', 'sowing_depth_cm', 'root_depth_cm', 'stress_tolerance_index', 'row_spacing_cm', 'plant_to_plant_cm', 'seed_per_spot', 'seed_rate_m2', 'seed_weight_kg_ha', 'germination_percentage', 'effective_plants_m2', 'tillers_per_plant', 'shoots_per_m2', 'survival_ratio', 'shoot_bearing_m2', 'leaf_area_per_shoot_m2', 'start_lai', 'lai_start_gdd', 'max_lai', 'lai_max_gdd', 'end_lai', 'lai_end_gdd', 'rue_max', 'rue_asymptote_frac', 'rue_lai_half_sat', 'rue_

,crop_fk,variety_eng,variety_type,height_class,height_cm,nutrient_content,common_disease_tolerance,common_pest_tolerance,environmental_resilience,category,...,sandy_loam,loam,silt_loam,clay_loam,silt,sandy_clay,silty_clay_loam,silty_clay,clay,sandy_clay_loam
0,wheat,Sadiq-21,OPV,Medium,115,Protein 12-13%,Leaf & Yellow Rust (MR),Aphid (Moderate),Drought Resistance,Cereal,...,1,1,1,1,0,0,0,0,1,0
1,wheat,aari-2011,OPV,Medium,105,Protein 12-13%,"Leaf rust, stripe rust",Termite,NaN,Cereal,...,1,1,1,1,0,0,0,0,1,0
2,wheat,aas-2011,OPV,Medium,105,Protein 12-13%,Rust resistant,Aphid,NaN,Cereal,...,1,1,1,1,0,0,0,0,1,0
3,wheat,akbar-2019,OPV,Medium,105,Protein 12-13%,Rust resistant (all 3 rusts),Aphid,NaN,Cereal,...,1,1,1,1,0,0,0,0,1,0
4,wheat,anaj-2017,OPV,Medium,105,Protein 12-13%,Rust resistant,Aphid,NaN,Cereal,...,1,1,1,1,0,0,0,0,1,0


In [4]:
# Put your exact desired order here (subset or superset is OK)
REQUIRED_ORDER = [
    # ---- identifiers / descriptors ----
    "crop_fk",
    "variety_eng",
    "variety_type",
    "country",
    "season",
    "region",
    # ---- sowing window ----
    "sowing_start_season",
    "sowing_end_season",
    # ---- seeds / weights ----
    "seed_weight_mg", 
    # "seed_weight_kg_ha", 
    # "seed_rate_m2",
    # ---- temperatures / days / gdd ----
    "min_temp",
    "base_temp",
    "opt_temp",
    "upper_temp",
    "min_days",
    "max_days", 
    "ripening_start_gdd",
    "ripening_end_gdd",
    # "avg_daily_gdd", "days_to_ripening",
    # ---- environment / weather ----
    # "par",
    "sunshine",
    "soil_temp_min",
    "vpa_max",
    "humidity_max",
    "dew_point_min",
    "ws_2m_max", 
    # "gdd_adjust",
    # ---- soil / water / ph ----
    "ph_min",
    "ph_max",
    "cwr_min",
    "cwr_max",
    # ---- agronomy ----
    "sowing_depth_cm",
    "root_depth_cm",
    "stress_tolerance_index",
    "row_spacing_cm",
    "plant_to_plant_cm",
    "seed_per_spot",
    "germination_percentage", 
    # "effective_plants_m2",
    "tillers_per_plant",
    # "shoots_per_m2",
    "survival_ratio", 
    # "shoot_bearing_per_m2",
    # ---- LAI / RUE ----
    "leaf_area_per_shoot_m2", 
    # "start_lai",
    "lai_start_gdd",
    # "max_lai",
    "lai_max_gdd",
    # "end_lai",
    "lai_end_gdd",
    "rue_max",
    "rue_asymptote_frac",
    "rue_lai_half_sat",
    "rue_daily_delta_max",
    # ---- yield ----
    "harvest_index",
    "yield_t_ha",
    "yield_type",
    # ---- BBCH ----
    "lai_start_bbch",
    "lai_max_bbch",
    "lai_end_bbch",
    # ---- DM: FM fruiting BBCH ----
    "fruiting_start_bbch",
    "dm_fm_ratio",
    # ---- nutrients / deficiencies ----
    "n_deficiency_kg_ha",
    "p_deficiency_kg_ha",
    "k_deficiency_kg_ha",
    # ---- traits / notes ----
    "height_class",
    "height_cm",
    "nutrient_content",
    "common_disease_tolerance",
    "common_pest_tolerance",
    "environmental_resilience",
    # ---- soil types ----
    "sand",
    "loamy_sand",
    "sandy_loam",
    "loam",
    "silt_loam",
    "clay_loam",
    "silt",
    "sandy_clay",
    "silty_clay_loam",
    "silty_clay",
    "clay",
    "sandy_clay_loam",
    # ---- batch timestamps ----
    "created_at", "updated_at",
]

# Ensure presence; create missing with NA
for col in REQUIRED_ORDER:
    if col not in df_renamed.columns:
        df_renamed[col] = pd.NA

# Put required columns first, keep any extras at the end (in their current order)
ordered = [c for c in REQUIRED_ORDER if c in df_renamed.columns]
extras  = [c for c in df_renamed.columns if c not in ordered]
df_ordered = df_renamed[ordered + extras].copy()

print("Final order (first 30 shown):", (ordered + extras)[:30], "...")
display(df_ordered.head())


Final order (first 30 shown): ['crop_fk', 'variety_eng', 'variety_type', 'country', 'season', 'region', 'sowing_start_season', 'sowing_end_season', 'seed_weight_mg', 'min_temp', 'base_temp', 'opt_temp', 'upper_temp', 'min_days', 'max_days', 'ripening_start_gdd', 'ripening_end_gdd', 'sunshine', 'soil_temp_min', 'vpa_max', 'humidity_max', 'dew_point_min', 'ws_2m_max', 'ph_min', 'ph_max', 'cwr_min', 'cwr_max', 'sowing_depth_cm', 'root_depth_cm', 'stress_tolerance_index'] ...


,crop_fk,variety_eng,variety_type,country,season,region,sowing_start_season,sowing_end_season,seed_weight_mg,min_temp,...,days_to_ripening,gdd_adjust,seed_rate_m2,seed_weight_kg_ha,effective_plants_m2,shoots_per_m2,shoot_bearing_m2,start_lai,max_lai,end_lai
0,wheat,Sadiq-21,OPV,Pakistan,Rabi,Drought,2023-10-01 00:00:00,2023-12-31 00:00:00,35,4,...,151.03701741751473,NaN,333.3333333333333,116.67,300,300,139.89999999999998,0.5,2.1,1.785
1,wheat,aari-2011,OPV,Pakistan,Rabi,irrigated,2023-10-01 00:00:00,2023-12-31 00:00:00,35,4,...,135.25116072886772,NaN,333.3333333333333,116.67,300,900,719.5826373626377,0.5,5.76,4.896
2,wheat,aas-2011,OPV,Pakistan,Rabi,irrigated,2023-10-01 00:00:00,2023-12-31 00:00:00,35,4,...,135.25116072886772,NaN,333.3333333333333,116.67,300,900,719.5826373626377,0.5,5.76,4.896
3,wheat,akbar-2019,OPV,Pakistan,Rabi,irrigated,2023-10-01 00:00:00,2023-12-31 00:00:00,35,4,...,135.25116072886772,NaN,333.3333333333333,116.67,300,900,719.5826373626377,0.5,5.76,4.896
4,wheat,anaj-2017,OPV,Pakistan,Rabi,irrigated,2023-10-01 00:00:00,2023-12-31 00:00:00,35,4,...,135.25116072886772,NaN,333.3333333333333,116.67,300,900,719.5826373626377,0.5,5.76,4.896


In [29]:
# cols_to_check = ["sand", "sandy_loam", "loamy_sand"]
# print(df_ordered[cols_to_check].dtypes)


In [5]:
import pandas as pd
from decimal import Decimal, ROUND_HALF_UP

# ---- classify columns ----
INT_COLS = [
    "min_days",
    "max_days",
    "ripening_start_gdd",
    "ripening_end_gdd",
    "seed_per_spot",
    "germination_percentage",
    "lai_start_gdd",
    "lai_max_gdd",
    "lai_end_gdd",
    "height_cm",
    "n_deficiency_kg_ha",
    "p_deficiency_kg_ha",
    "k_deficiency_kg_ha",
]

NUMERIC_2DP = [
    "seed_weight_mg",
    "min_temp",
    "base_temp",
    "opt_temp",
    "upper_temp",
    "sunshine",
    "soil_temp_min",
    "vpa_max",
    "humidity_max",
    "dew_point_min",
    "ws_2m_max",
    "ph_min",
    "ph_max",
    "cwr_min",
    "cwr_max",
    "root_depth_cm",
    "sowing_depth_cm",
    "stress_tolerance_index",
    "row_spacing_cm",
    "plant_to_plant_cm",
    "tillers_per_plant",
    "survival_ratio",
    "rue_max",
    "rue_asymptote_frac",
    "rue_lai_half_sat",
    "harvest_index",
    "yield_t_ha",
    "rue_daily_delta_max",
    "dm_fm_ratio"
]

NUMERIC_3DP = [
    "leaf_area_per_shoot_m2"
]

DATE_COLS = [
    "sowing_start_season",
    "sowing_end_season"
    ]

TS_COLS = [
    "created_at",
    "updated_at"
    ]

BOOL_COLS = [
    "sand",
    "loamy_sand",
    "sandy_loam",
    "loam",
    "silt_loam",
    "clay_loam",
    "silt",
    "sandy_clay",
    "silty_clay_loam",
    "silty_clay",
    "clay",
    "sandy_clay_loam",
]

data = df_ordered.copy()

# ---- normalize strings ----
string_cols = [c for c in data.columns if c not in INT_COLS + NUMERIC_2DP + NUMERIC_3DP + TS_COLS + BOOL_COLS]
for c in string_cols:
    data[c] = data[c].astype("string").str.strip().str.lower()

# Fix lai_*_gdd columns before casting
for c in ["lai_max_gdd", "lai_end_gdd"]:
    if c in data.columns:
        # Extract only the numeric part from strings like "65: Full Flowering..."
        data[c] = (
            data[c]
            .astype(str)
            .str.extract(r"(\d+)", expand=False)  # keep only first group of digits
        )

# ---- integers ----
for c in INT_COLS:
    if c in data.columns:
        data[c] = pd.to_numeric(data[c], errors="coerce").astype("Int64")

# ---- helper to fix scale with Decimal ----
def to_decimal_series(s: pd.Series, places: int):
    q = Decimal("0." + "0"*places) if places > 0 else Decimal("0")
    return (pd.to_numeric(s, errors="coerce")
              .round(places)                              # numeric rounding (optional)
              .astype("object")                           # switch out of float dtype
              .map(lambda x: None if pd.isna(x) else Decimal(str(x)).quantize(q, rounding=ROUND_HALF_UP)))

# ---- 2dp & 3dp numerics as Decimal ----
for c in NUMERIC_2DP:
    if c in data.columns:
        data[c] = to_decimal_series(data[c], 2)

for c in NUMERIC_3DP:
    if c in data.columns:
        data[c] = to_decimal_series(data[c], 3)

# convert date-like columns to Python date (not string)
for c in DATE_COLS:
    if c in data.columns:
        dt = pd.to_datetime(data[c], errors="coerce", utc=False)  # e.g. '2023-10-01 00:00:00'
        data[c] = dt.dt.date  # -> Python date objects (works with SQLAlchemy Date)

# ---- booleans ----
for c in BOOL_COLS:
    if c in data.columns:
        # 1 → True; blank/NaN → False
        data[c] = (
            data[c]
            .apply(lambda x: True if str(x).strip() == "1" else False)
            .astype("boolean")
        )

# ---- timestamps ----
current_ts = pd.Timestamp.now(tz="UTC")
data["created_at"] = current_ts
data["updated_at"] = current_ts

# ---- ensure NULLs ----
data = data.where(pd.notna(data), None)

print("✅ Cleaning complete. dtypes summary:")
display(data.dtypes)
display(data.head())


✅ Cleaning complete. dtypes summary:


crop_fk             string[python]
variety_eng         string[python]
variety_type        string[python]
country             string[python]
season              string[python]
                         ...      
shoots_per_m2       string[python]
shoot_bearing_m2    string[python]
start_lai           string[python]
max_lai             string[python]
end_lai             string[python]
Length: 88, dtype: object

,crop_fk,variety_eng,variety_type,country,season,region,sowing_start_season,sowing_end_season,seed_weight_mg,min_temp,...,days_to_ripening,gdd_adjust,seed_rate_m2,seed_weight_kg_ha,effective_plants_m2,shoots_per_m2,shoot_bearing_m2,start_lai,max_lai,end_lai
0,wheat,sadiq-21,opv,pakistan,rabi,drought,2023-10-01,2023-12-31,35.00,4.00,...,151.03701741751473,<NA>,333.3333333333333,116.67,300,300,139.89999999999998,0.5,2.1,1.785
1,wheat,aari-2011,opv,pakistan,rabi,irrigated,2023-10-01,2023-12-31,35.00,4.00,...,135.25116072886772,<NA>,333.3333333333333,116.67,300,900,719.5826373626377,0.5,5.76,4.896
2,wheat,aas-2011,opv,pakistan,rabi,irrigated,2023-10-01,2023-12-31,35.00,4.00,...,135.25116072886772,<NA>,333.3333333333333,116.67,300,900,719.5826373626377,0.5,5.76,4.896
3,wheat,akbar-2019,opv,pakistan,rabi,irrigated,2023-10-01,2023-12-31,35.00,4.00,...,135.25116072886772,<NA>,333.3333333333333,116.67,300,900,719.5826373626377,0.5,5.76,4.896
4,wheat,anaj-2017,opv,pakistan,rabi,irrigated,2023-10-01,2023-12-31,35.00,4.00,...,135.25116072886772,<NA>,333.3333333333333,116.67,300,900,719.5826373626377,0.5,5.76,4.896


In [6]:
from sqlalchemy import MetaData, Table, Column, text
from sqlalchemy import String, Text, Integer, DateTime, Numeric, Date, Boolean

metadata = MetaData(schema=PG_SCHEMA)

table = Table(
    TABLE_NAME, metadata,
    # identifiers
    Column("variety_eng",               String, primary_key=True),
    Column("variety_type",              String),
    Column("country",                   String),
    Column("season",                    String),
    Column("sowing_start_season",       Date),
    Column("sowing_end_season",         Date),
    Column("region",                    String),
    Column("seed_weight_mg",            Numeric(10, 2)),
    Column("min_temp",                  Numeric(6, 2)),
    Column("base_temp",                 Numeric(6, 2)),
    Column("opt_temp",                  Numeric(6, 2)),
    Column("upper_temp",                Numeric(6, 2)),
    Column("min_days",                  Integer),
    Column("max_days",                  Integer),
    Column("ripening_start_gdd",        Integer),
    Column("ripening_end_gdd",          Integer),
    Column("sunshine",                  Numeric(6, 2)),
    Column("soil_temp_min",             Numeric(6, 2)),
    Column("vpa_max",                   Numeric(6, 2)),
    Column("humidity_max",              Numeric(6, 2)),
    Column("dew_point_min",             Numeric(6, 2)),
    Column("ws_2m_max",                 Numeric(6, 2)),
    Column("ph_min",                    Numeric(4, 2)),
    Column("ph_max",                    Numeric(4, 2)),
    Column("cwr_min",                   Integer),
    Column("cwr_max",                   Integer),
    Column("sowing_depth_cm",           Numeric(8, 2)),
    Column("root_depth_cm",             Numeric(8, 2)),
    Column("stress_tolerance_index",    Numeric(6, 2)),
    Column("row_spacing_cm",            Numeric(8, 2)),
    Column("plant_to_plant_cm",         Numeric(8, 2)),
    Column("seed_per_spot",             Integer),
    Column("germination_percentage",    Integer),
    Column("tillers_per_plant",         Numeric(6, 2)),
    Column("survival_ratio",            Numeric(8, 3)),
    Column("leaf_area_per_shoot_m2",    Numeric(8, 3)),
    Column("lai_start_gdd",             Integer),
    Column("lai_max_gdd",               Integer),
    Column("lai_end_gdd",               Integer),
    Column("rue_max",                   Numeric(6, 2)),
    Column("rue_asymptote_frac",        Numeric(6, 2)),
    Column("rue_lai_half_sat",          Numeric(6, 2)),
    Column("rue_daily_delta_max",       Numeric(8, 3)),
    Column("harvest_index",             Numeric(6, 2)),
    Column("yield_t_ha",                Numeric(10, 2)),
    Column("yield_type",                String),
    Column("lai_start_bbch",            String),
    Column("lai_max_bbch",              String),
    Column("lai_end_bbch",              String),
    Column("fruiting_start_bbch",       String),
    Column("dm_fm_ratio",               Numeric(4, 2)),
    Column("n_deficiency_kg_ha",        Integer),
    Column("p_deficiency_kg_ha",        Integer),
    Column("k_deficiency_kg_ha",        Integer),
    Column("height_class",              String),
    Column("height_cm",                 Integer),
    Column("nutrient_content",          Text),
    Column("common_disease_tolerance",  Text),
    Column("common_pest_tolerance",     Text),
    Column("environmental_resilience",     Text),
    Column("sand",                      Boolean),
    Column("loamy_sand",                Boolean),
    Column("sandy_loam",                Boolean),
    Column("loam",                      Boolean),
    Column("silt_loam",                 Boolean),
    Column("clay_loam",                 Boolean),
    Column("silt",                      Boolean),
    Column("sandy_clay",                Boolean),
    Column("silty_clay_loam",           Boolean),
    Column("silty_clay",                Boolean),
    Column("clay",                      Boolean),
    Column("sandy_clay_loam",           Boolean),
    Column("crop_fk",                   Text),
    Column("created_at",                DateTime(timezone=True)),
    Column("updated_at",                DateTime(timezone=True)),
)

with engine.begin() as conn:
    conn.execute(text(f'CREATE SCHEMA IF NOT EXISTS "{PG_SCHEMA}";'))
    metadata.create_all(conn)
    print(f"✅ Ensured table {PG_SCHEMA}.{TABLE_NAME} exists (or was created).")


✅ Ensured table lgs2.crop_varieties exists (or was created).


In [7]:
from sqlalchemy import String as SA_String, Text as SA_Text, Integer as SA_Integer, DateTime as SA_DateTime, Numeric as SA_Numeric, Date as SA_Date, Boolean as SA_Boolean

dtype_map = {
    "variety_eng":                  SA_String(),
    "variety_type":                 SA_String(),
    "country":                      SA_String(),
    "season":                       SA_String(),
    "sowing_start_season":          SA_Date(),
    "sowing_end_season":            SA_Date(),
    "region":                       SA_String(),
    "seed_weight_mg":               SA_Numeric(10, 2),
    "min_temp":                     SA_Numeric(6, 2),
    "base_temp":                    SA_Numeric(6, 2),
    "opt_temp":                     SA_Numeric(6, 2),
    "upper_temp":                   SA_Numeric(6, 2),
    "min_days":                     SA_Integer(),
    "max_days":                     SA_Integer(),
    "ripening_start_gdd":           SA_Integer(),
    "ripening_end_gdd":             SA_Integer(),
    "sunshine":                     SA_Numeric(6, 2),
    "soil_temp_min":                SA_Numeric(6, 2),
    "vpa_max":                      SA_Numeric(6, 2),
    "humidity_max":                 SA_Numeric(6, 2),
    "dew_point_min":                SA_Numeric(6, 2),
    "ws_2m_max":                    SA_Numeric(6, 2),
    "ph_min":                       SA_Numeric(4, 2),
    "ph_max":                       SA_Numeric(4, 2),
    "cwr_min":                      SA_Integer(),
    "cwr_max":                      SA_Integer(),
    "sowing_depth_cm":              SA_Numeric(8, 2),
    "root_depth_cm":                SA_Numeric(8, 2),
    "stress_tolerance_index":       SA_Numeric(6, 2),
    "row_spacing_cm":               SA_Numeric(8, 2),
    "plant_to_plant_cm":            SA_Numeric(8, 2),
    "seed_per_spot":                SA_Integer(), 
    "germination_percentage":       SA_Integer(), 
    "tillers_per_plant":            SA_Numeric(6, 2),
    "survival_ratio":               SA_Numeric(8, 3),
    "leaf_area_per_shoot_m2":       SA_Numeric(8, 3),
    "lai_start_gdd":                SA_Integer(), 
    "lai_max_gdd":                  SA_Integer(), 
    "lai_end_gdd":                  SA_Integer(), 
    "rue_max":                      SA_Numeric(6, 2),
    "rue_asymptote_frac":           SA_Numeric(6, 2),
    "rue_lai_half_sat":             SA_Numeric(6, 2),
    "rue_daily_delta_max":          SA_Numeric(8, 3),
    "harvest_index":                SA_Numeric(6, 2),
    "yield_t_ha":                   SA_Numeric(10, 2),
    "yield_type":                   SA_String(),
    "lai_start_bbch":               SA_String(),
    "lai_max_bbch":                 SA_String(),
    "lai_end_bbch":                 SA_String(),
    "fruiting_start_bbch":          SA_String(),
    "dm_fm_ratio":                  SA_Numeric(4,2),
    "n_deficiency_kg_ha":           SA_Integer(),
    "p_deficiency_kg_ha":           SA_Integer(),
    "k_deficiency_kg_ha":           SA_Integer(),
    "height_class":                 SA_String(),
    "height_cm":                    SA_Integer(),
    "nutrient_content":             SA_Text(),
    "common_disease_tolerance":     SA_Text(),
    "common_pest_tolerance":        SA_Text(),
    "environmental_resilience":              SA_Text(),
    "sand":                         SA_Boolean(),
    "loamy_sand":                   SA_Boolean(),
    "sandy_loam":                   SA_Boolean(),
    "loam":                         SA_Boolean(),
    "silt_loam":                    SA_Boolean(),
    "clay_loam":                    SA_Boolean(),
    "silt":                         SA_Boolean(),
    "sandy_clay":                   SA_Boolean(),
    "silty_clay_loam":              SA_Boolean(),
    "silty_clay":                   SA_Boolean(),
    "clay":                         SA_Boolean(),
    "sandy_clay_loam":              SA_Boolean(),
    "crop_fk":                      SA_Text(),
    "created_at":                   SA_DateTime(timezone=True),
    "updated_at":                   SA_DateTime(timezone=True),
}


In [9]:
# --- keep only the target crop before any ordering/dedup/upload ---
TARGET_CROP = "wheat"

if "crop_fk" not in data.columns:
    raise KeyError("Expected column 'crop_fk' not found in DataFrame")

# normalize and filter
crop_norm = data["crop_fk"].astype("string").str.strip().str.casefold()
mask = (crop_norm == TARGET_CROP.casefold())

total_rows = len(data)
kept_rows  = int(mask.sum())

if kept_rows == 0:
    # hard stop so you don't accidentally upload other crops
    raise ValueError(f"No rows matched crop_fk = '{TARGET_CROP}'. Aborting upload.")
elif kept_rows < total_rows:
    others = (
        data.loc[~mask, "crop_fk"]
        .dropna()
        .astype("string")
        .str.strip()
        .str.casefold()
        .unique()
        .tolist()
    )
    print(f"ℹ️ Filtering to '{TARGET_CROP}'. Kept {kept_rows}/{total_rows} rows. "
          f"Other crop_fk values present (excluded): {others}")

# apply filter
data = data.loc[mask].copy()

# (optional) assert everything left is the target crop
assert set(
    data["crop_fk"].astype("string").str.strip().str.casefold().unique()
) <= {TARGET_CROP.casefold()}, "Found non-target crop rows after filtering."

# --- proceed with your existing ordering/dedup/upload ---
# order by variety_eng (case-insensitive), stable
data["variety_eng"] = data["variety_eng"].astype("string")

data = (
    data.sort_values(
        by="variety_eng",
        key=lambda s: s.fillna("").str.casefold(),
        kind="mergesort",
        na_position="last",
    )
    .drop_duplicates(subset=["variety_eng"], keep="last")
    .reset_index(drop=True)
)

# --- drop any columns you don’t want in DB (already handled above if needed) ---
for drop_col in [
    "category", "par", "avg_daily_gdd",
    "days_to_ripening", "gdd_adjust", "seed_rate_m2", "seed_weight_kg_ha",
    "effective_plants_m2", "shoots_per_m2", "shoot_bearing_m2",
    "start_lai", "max_lai", "end_lai"
    ]:
    if drop_col in data.columns:
        print(f"🧹 Dropping '{drop_col}' from DataFrame before upload")
        data = data.drop(columns=[drop_col])

# --- upload ---
data.to_sql(
    name=TABLE_NAME,
    con=engine,
    schema=PG_SCHEMA,
    if_exists="append",
    index=False,
    chunksize=10_000,
    method="multi",
    dtype=dtype_map,
)

with engine.connect() as conn:
    count = conn.execute(text(f'SELECT COUNT(*) FROM "{PG_SCHEMA}"."{TABLE_NAME}";')).scalar_one()
print(f"✅ Upload complete (crop='{TARGET_CROP}'). Row count in {PG_SCHEMA}.{TABLE_NAME}: {count}")


🧹 Dropping 'par' from DataFrame before upload
✅ Upload complete (crop='wheat'). Row count in lgs2.crop_varieties: 26


In [52]:
# Pandas-side dtypes (should match what we intended)
print("Pandas dtypes:")
display(data.dtypes)

# DB-side types (optional: inspect)
qry = f"""
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_schema = :schema AND table_name = :table
ORDER BY ordinal_position;
"""
with engine.connect() as conn:
    rows = conn.execute(text(qry), {"schema": PG_SCHEMA, "table": TABLE_NAME}).mappings().all()
pd.DataFrame(rows)


Pandas dtypes:


crop_fk                 string[python]
variety_eng             string[python]
variety_type            string[python]
country                 string[python]
season                  string[python]
                          ...         
silty_clay                     boolean
clay                           boolean
sandy_clay_loam                boolean
created_at         datetime64[us, UTC]
updated_at         datetime64[us, UTC]
Length: 76, dtype: object

,column_name,data_type,is_nullable
0,variety_eng,character varying,NO
1,variety_type,character varying,YES
2,country,character varying,YES
3,season,character varying,YES
4,sowing_start_season,date,YES
...,...,...,...
71,clay,boolean,YES
72,sandy_clay_loam,boolean,YES
73,crop_fk,text,YES
74,created_at,timestamp with time zone,YES
